# Code

## Imports and packages required

In [1]:
import numpy as np
import pandas as pd

## Preprocessing

### 1. Reading the CSV File

In [2]:
df = pd.read_csv('../data/online_retail_listing.csv', on_bad_lines='skip',delimiter=';', decimal = ',')
df.head(3)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,1.12.2009 07:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,1.12.2009 07:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,1.12.2009 07:45,6.75,13085.0,United Kingdom


### 2. InvoiceDate column type conversion from String to DateTime

In [3]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format = "%d.%m.%Y %H:%M")

### 3. Customer ID column type conversion from Float to Int

In [4]:
df['Customer ID'] = df['Customer ID'].astype('Int64')

### 4. Quantity and price - Filter out bad values

In [5]:
df = df[df['Quantity']>=0]
df = df[df['Price']>=0]

### 5. Processing the Description

1. Convert all descriptions to uppercase
2. Find `StockCode`s with multiple descriptions (Lengthy, Short, NaN, etc.) and keep the longest string as the detailed description
3. Filter out transactions with no descriptions (66 found)
4. Strip the string to remove any leading and trailing whitespaces

In [6]:
#1
df["Description"] = df["Description"].str.upper()

#2
grps = df[["StockCode", "Description"]].drop_duplicates().groupby("StockCode")
desc_replacer = dict()
for grp in grps:
    if len(grp[1])>1:
        desc_replacer[grp[0]] = grp[1]["Description"][grp[1]["Description"].map(str).agg(len).idxmax()]

df["Description"] = df.apply(
        axis=1, 
        func=lambda x: desc_replacer[x["StockCode"]] if x["StockCode"] in desc_replacer.keys() else x["Description"]
    )

#3
df = df[df['Description'].notna()]

#4
df["Description"] = df["Description"].map(lambda x: str(x).strip())

### 6. Processing the Customer IDs

In the dataset, we know that there is no such entry where the same invoice is shared between two customers.

Let us check if the same invoice has a customer ID in some entries, and NA in other

In [7]:
df["Customer ID"].isna().sum()

232427

In [8]:
empty_CID_Orders = df[df["Customer ID"].isna()]["Invoice"].drop_duplicates().values.tolist()
display(df[df["Invoice"].map(lambda x: x in empty_CID_Orders)]["Customer ID"].notna().sum())

0

Since there are no invoices where we can recover `Customer ID` from another entry, we can give placeholder IDs to Customers with ID `NA`, assuming each invoice corresponds to a new customer.

In [9]:
max_existing_id = df["Customer ID"].max()
invoice_cust_dict = dict()

for key in empty_CID_Orders:
    max_existing_id += 1
    invoice_cust_dict[key] = max_existing_id

df["Valid_Customer"] = df["Customer ID"].notna().map(lambda x: int(x))
df["Customer ID"] = df.apply(
        axis=1, 
        func=lambda x: invoice_cust_dict[x["Invoice"]] if x["Invoice"] in invoice_cust_dict.keys() else x["Customer ID"]
    )

### 7. Removing Cancelled transactions

In [14]:
df = df[df["Invoice"].map(lambda x: "c" not in str(x).lower())]

### 8. Drop duplicate entries of the same transaction

In [15]:
df.drop_duplicates(inplace=True)

### 9. Check for invalid data in the DataFrame

In [16]:
df.isna().sum()

Invoice           0
StockCode         0
Description       0
Quantity          0
InvoiceDate       0
Price             0
Customer ID       0
Country           0
Valid_Customer    0
dtype: int64

### 10. Save the cleaned DataFrame for further analysis

In [17]:
df.to_csv('../data/online_retail_listing_cleaned.csv', index=False)

## Exploratory Data Analysis

In [ ]:
#!pip install pandas-profiling

In [ ]:
import pandas_profiling
#Generating PandasProfiling Report
report = pandas_profiling.ProfileReport(df)

In [ ]:
report

In [ ]:
df.drop_duplicates()

In [ ]:
# skewness along the index axis
df.skew(axis = 0, skipna = True)# highly skewed price, indicating there might be a few low priced items 

In [ ]:
df['Country'].unique()


## Market Basket Analysis

Unstack the df into basket with invoice and product descriptions

In [ ]:
basket = df.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('Invoice')

In [ ]:
basket 

Encode 0.0 to 0 and 1.0 to 1

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
basket_encode_plus = basket.applymap(encode_units)
basket_encode_plus

Filter out Invoices with atleast 2 products 

In [ ]:
basket_filter_plus = basket_encode_plus[(basket_encode_plus > 0).sum(axis=1) >= 2]
basket_filter_plus

In [ ]:
#!pip install mlxtend

In [ ]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets_plus = apriori(basket_filter_plus, min_support=0.03, 
                                 use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

frequent_itemsets_plus['length'] = frequent_itemsets_plus['itemsets'].apply(lambda x: len(x))

frequent_itemsets_plus

In [ ]:
from mlxtend.frequent_patterns import association_rules

In [ ]:
association_rules(frequent_itemsets_plus, metric='lift', 
                  min_threshold=1).sort_values('lift', ascending=False).reset_index(drop=True)

## Customer Segmentation

## Market basket Analysis - Chyavan

#### Complementary Products

In [2]:
df_mba = pd.read_csv("../data/online_retail_listing_cleaned.csv")
df_mba['InvoiceDate'] = pd.to_datetime(df_mba['InvoiceDate'], format = "%Y-%m-%d %H:%M:%S")
df_mba.head()

C:\Users\chyav\AppData\Local\Temp\ipykernel_16684\3878480219.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mba = pd.read_csv("../data/online_retail_listing_cleaned.csv")


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Valid_Customer
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,1
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,1
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,1
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,1
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,1


In [3]:
mba = df_mba[["Invoice", "StockCode"]]
unique_items = mba["StockCode"].drop_duplicates()

In [4]:
mba_unique = mba.drop_duplicates()
mba_crosstab = pd.crosstab(mba_unique["Invoice"], mba_unique["StockCode"])

product_counts = unique_items.map(lambda product: mba_crosstab[[product]].value_counts().get(1, 0))
product_counts.index = unique_items.values

N = len(mba_crosstab)

In [5]:
def calculate_lift(product_1, product_2):
    count_1 = product_counts[product_1]
    count_2 = product_counts[product_2]
    count_12 = mba_crosstab[[product_1, product_2]].value_counts().get((1,1), 0)
    if count_12 == 0:
        return 0
    return N*count_12/(count_1*count_2)

calculate_lift("85048", "23562")

3.9035336356764927

In [54]:
# 1.  Load Lift Matrix
try:
    lifts = pd.read_csv("../data/lifts.csv", index_col="idx")
    lifts.index.name = None

# 2. Create Lift Matrix
except FileNotFoundError: 
    lifts = pd.DataFrame([], columns=unique_items.values, index=unique_items.values)
    lifts["processed"] = False

lifts.head()

,85048,79323P,79323W,22041,21232,22064,21871,21523,22350,22349,...,23578,23580,23575,23560,23576,23562,23561,23609,23617,processed
85048,NaN,3.093366,2.818597,2.402175,2.238435,3.269281,3.624347,2.064099,3.209833,2.239732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
79323P,3.093366,NaN,72.815538,1.427708,3.096182,7.138959,6.284884,5.467914,4.695954,3.833746,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
79323W,2.818597,72.815538,NaN,1.300891,2.889733,3.846948,4.618248,5.003517,4.635404,3.881347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
22041,2.402175,1.427708,1.300891,NaN,3.882041,4.470812,4.723131,2.789936,3.190840,2.274190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
21232,2.238435,3.096182,2.889733,3.882041,NaN,10.857026,4.001080,2.828422,3.140674,2.241771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [53]:
import datetime

start = datetime.datetime.now()

for i, item1 in enumerate(unique_items.values):
    if lifts.iloc[i]["processed"]:
        continue
    for j, item2 in enumerate(unique_items.values):
        if i > j:
            lift = calculate_lift(item1, item2)
            lifts.loc[item1, item2] = lift
            lifts.loc[item2, item1] = lift
    lifts.loc[item1, "processed"] = True
    lifts.to_csv("../data/lifts.csv", index_label="idx")
    end = datetime.datetime.now()
    if (end - start).seconds > 25200:
        break

In [61]:
# mba_crosstab[["85048", "23562"]].value_counts().get((1,1), 0)
# mba_crosstab[["21232"]].value_counts().get(1, 0)

#### Substitutive Products